In [1]:
import os
%load_ext rpy2.ipython


In [2]:
%%R
workDir = '/home/chantal/Chazy/data/'
figDir = '/home/chantal/Chazy/Bulk/ITS/data/figs/'

physeqDir = '/home/chantal/Chazy/data/phyloseq/ITS/'
physeqBulk = 'bulk-core'


nprocs = 20

In [3]:
%%R
library(ggplot2)
library(phyloseq)
library(tidyr)
library(plyr);library(dplyr)
library(scales)
library(biom)
library(metagenomeSeq)
library(doParallel)


/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘dplyr’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/opt/anaconda/lib/python2.7/site-packages/rpy2/robjects/f

In [4]:
%%R
F = file.path(physeqDir, physeqBulk)
physeq.Bulk = readRDS(F)
physeq.Bulk.m = physeq.Bulk %>% sample_data
#physeq.Bulk.Sparsity = filter_taxa(physeq.Bulk, function(x) sum(x > 0) > (0.25 * length(x)), TRUE)

physeq.Bulk.m = physeq.Bulk %>% sample_data

physeq.Bulk.core = prune_samples(physeq.Bulk.m$Land_Management != "PAS", physeq.Bulk)

physeq.Bulk.core = physeq.Bulk.core %>% filter_taxa(function(x) sum(x) > 0, TRUE)

physeq.Bulk.m = physeq.Bulk.core %>% sample_data
    

#print(physeq.Bulk.Sparsity)
print(physeq.Bulk.core)


phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 2232 taxa and 176 samples ]
sample_data() Sample Data:       [ 176 samples by 24 sample variables ]
tax_table()   Taxonomy Table:    [ 2232 taxa by 7 taxonomic ranks ]


In [5]:
%%R
physeq.Bulk.m = physeq.Bulk %>% sample_data

In [6]:
%%R
physeq.Bulk.m$Sample_Date = as.POSIXct(strptime(physeq.Bulk.m$Sample_Date, format = "%m/%d/%y"))
str(physeq.Bulk.m$Sample_Date)

 POSIXct[1:185], format: "2015-04-30" "2015-08-13" "2015-04-30" "2015-07-16" ...


In [7]:
%%R
physeq.Bulk.m$Day = round(difftime(physeq.Bulk.m$Sample_Date, min(physeq.Bulk.m$Sample_Date), 
         units = "days"))
tail(physeq.Bulk.m)

Sample Data:        [6 samples by 24 sample variables]:
                    Full.sample Fraction PrimerF PrimerR Barcode Rev.index
Sept.NTR.2.2015 Sept.NTR.2.2015       NA  nBITSf   58A2r     144  gtaacgag
Aug.PTH.2.2015   Aug.PTH.2.2015       NA  nBITSf   58A2r     122  tctctatg
Aug.PAS.2015       Aug.PAS.2015       NA  nBITSf   58A2r     134  gatctacg
Sept.NTH.1.2015 Sept.NTH.1.2015       NA  nBITSf   58A2r     136  gatctacg
Aug.NTR.4.2015   Aug.NTR.4.2015       NA  nBITSf   58A2r     133  gatctacg
Sept.NTH.2.2015 Sept.NTH.2.2015       NA  nBITSf   58A2r     142  gtaacgag
                Fwd.index Treatment      Day Pool     Experiment Gradient
Sept.NTR.2.2015  gacaccgt       NTR 440 days    5 Chazy.ITS.Bulk     <NA>
Aug.PTH.2.2015   actatctg       PTH 399 days    5 Chazy.ITS.Bulk     <NA>
Aug.PAS.2015     cgtgagtg       PAS 399 days    5 Chazy.ITS.Bulk     <NA>
Sept.NTH.1.2015  gacaccgt       NTH 440 days    5 Chazy.ITS.Bulk     <NA>
Aug.NTR.4.2015   tcatcgag       NTR 399 days    5

In [8]:
%%R
physeq.Bulk.m$Day_rel = as.numeric(physeq.Bulk.m$Day)


In [9]:
%%R
head(physeq.Bulk.m)


Sample Data:        [6 samples by 25 sample variables]:
                  Full.sample Fraction PrimerF PrimerR Barcode Rev.index
April.NTR.4       April.NTR.4       NA  nBITSf   58A2r     155  atagtacc
Aug.NTR.3.2015 Aug.NTR.3.2015       NA  nBITSf   58A2r     129  gatctacg
April.PTR.3       April.PTR.3       NA  nBITSf   58A2r     150  acgtgcgc
July15.NTH.3     July15.NTH.3       NA  nBITSf   58A2r     184  aacgctga
Nov.NTR.1           Nov.NTR.1       NA  nBITSf   58A2r     125  tctctatg
June.NTH.2         June.NTH.2       NA  nBITSf   58A2r     162  gcgtatac
               Fwd.index Treatment      Day Pool     Experiment Gradient
April.NTR.4     tagcgagt       NTR 294 days    4 Chazy.ITS.Bulk     <NA>
Aug.NTR.3.2015  atcgtacg       NTR 399 days    5 Chazy.ITS.Bulk     <NA>
April.PTR.3     cgtgagtg       PTR 294 days    4 Chazy.ITS.Bulk     <NA>
July15.NTH.3    gacaccgt       NTH 371 days    4 Chazy.ITS.Bulk     <NA>
Nov.NTR.1       tcatcgag       NTR 139 days    4 Chazy.ITS.Bulk     

In [10]:
%%R
physeq.final = merge_phyloseq(physeq.Bulk.core, physeq.Bulk.m)
physeq.final

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 2232 taxa and 176 samples ]
sample_data() Sample Data:       [ 176 samples by 25 sample variables ]
tax_table()   Taxonomy Table:    [ 2232 taxa by 7 taxonomic ranks ]


In [11]:
%%R
physeq.final

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 2232 taxa and 176 samples ]
sample_data() Sample Data:       [ 176 samples by 25 sample variables ]
tax_table()   Taxonomy Table:    [ 2232 taxa by 7 taxonomic ranks ]


In [12]:
%%R
make_metagenomeSeq = function(physeq) {
    require("metagenomeSeq")
    require("phyloseq")
    # Enforce orientation
    if (!taxa_are_rows(physeq)) {
        physeq <- t(physeq)
    }
    OTU = as(otu_table(physeq), "matrix")
    #OTUTill = subset(OTU, rownames(OTU) %in% df.Till.r$OTU)

    # Convert sample_data to AnnotatedDataFrame
    ADF = AnnotatedDataFrame(data.frame(sample_data(physeq)))
    # define dummy 'feature' data for OTUs, using their name Helps with
    # extraction and relating to taxonomy later on.
    TDF = AnnotatedDataFrame(data.frame(tax_table(physeq)))
    #TDFTill = subset(TDF, rownames(TDF) %in% df.Till.r$OTU)
    TDF$Rank9 = rownames(TDF)


    # Create the metagenomeSeq object
    MGS = newMRexperiment(counts = OTU, phenoData = ADF, featureData = TDF)
    # Trigger metagenomeSeq to calculate its Cumulative Sum scaling factor.
    MGS = cumNorm(MGS)
    return(MGS)
}

MR = make_metagenomeSeq(physeq.final)

In [13]:
%%R
str(MR)

Formal class 'MRexperiment' [package "metagenomeSeq"] with 8 slots
  ..@ expSummary       :<environment: 0xfd24760> 
  ..@ assayData        :<environment: 0xfb94158> 
  ..@ phenoData        :Formal class 'AnnotatedDataFrame' [package "Biobase"] with 4 slots
  .. .. ..@ varMetadata      :'data.frame':	25 obs. of  1 variable:
  .. .. .. ..$ labelDescription: chr [1:25] NA NA NA NA ...
  .. .. ..@ data             :'data.frame':	176 obs. of  25 variables:
  .. .. .. ..$ Full.sample       : Factor w/ 176 levels "April.NTH.1.2015",..: 8 17 14 35 73 50 2 3 129 63 ...
  .. .. .. ..$ Fraction          : int [1:176] NA NA NA NA NA NA NA NA NA NA ...
  .. .. .. ..$ PrimerF           : Factor w/ 1 level "nBITSf": 1 1 1 1 1 1 1 1 1 1 ...
  .. .. .. ..$ PrimerR           : Factor w/ 1 level "58A2r": 1 1 1 1 1 1 1 1 1 1 ...
  .. .. .. ..$ Barcode           : int [1:176] 155 129 150 184 125 162 145 149 99 168 ...
  .. .. .. ..$ Rev.index         : Factor w/ 16 levels "aacgctga","acgtgcgc",..: 4 8 2 1

In [14]:
%%R
TimeSeries = function(MR, feature) {  
    
    res = fitTimeSeries(obj = MR, lvl = 'Rank9', feature = feature, class = "Till",
                    id = "Full.sample", time = "Day_rel", log = TRUE)

    return(res)
}




In [15]:
%%R
OTU = as(otu_table(physeq.final), "matrix")

feature = rownames(OTU)
str(feature)

 chr [1:2232] "OTU.71996" "OTU.20472" "OTU.7528" "OTU.14609" ...


In [16]:
%%R 
mdf = psmelt(physeq.final)

In [ ]:
%%R
test = mdf %>% group_by(OTU) %>% summarise(sum = sum(Abundance))
range(test$sum)

[1]     1 69414


In [ ]:
%%R
#OTUs not found across all timepoints?
registerDoParallel(10)

TS = llply(feature, 
          TimeSeries, 
          .parallel = TRUE,
          MR = MR 
         )

In [75]:
%%R
str(TS)


Error in str(TS) : object 'TS' not found


In [ ]:
%%R
names(TS) = feature

In [ ]:
%%R
timeSeriesFits = sapply(TS,function(i){i[[1]]})[-grep("No",TS)]

In [ ]:
%%R
str(timeSeriesFits)

In [ ]:
%%R
head(timeSeriesFits[[2]])

In [ ]:
%%R
for(i in 1:length(timeSeriesFits)){
    rownames(timeSeriesFits[[i]]) =
    paste(
    paste(names(timeSeriesFits)[i]," interval",sep=""),
    1:nrow(timeSeriesFits[[i]]),sep=":"
)
}

In [ ]:
%%R
timeSeriesFits = as.data.frame(do.call(rbind, timeSeriesFits))


#do.call(rbind,timeSeriesFits)

In [ ]:
%%R
pvalues = timeSeriesFits[,"p.value"]
adjPvalues = p.adjust(pvalues,"bonferroni")
timeSeriesFits = cbind(timeSeriesFits,adjPvalues)
head(timeSeriesFits)

In [ ]:
%%R
write.csv(timeSeriesFits, 'data/timeSeries_Till_ITS.csv')

In [ ]:
%%R
TSOM = read.csv('data/timeSeries_Till_ITS.csv')

In [ ]:
%%R
TS.sig = filter(TSOM, adjPvalues <= 0.01)

In [ ]:
%%R
TS.sig$OTU_interval = TS.sig$X
TS.sig = separate(TS.sig, OTU_interval, c('OTU', 'interval'), sep = " ") %>% separate(interval, c('interval', 'num'))
head(TS.sig)

In [ ]:
%%R
print(length(unique(TS.sig$OTU))) 
Mult_resp = TS.sig %>% group_by(OTU) %>% summarise(respTime = n()) %>% filter(respTime > 1)

In [ ]:
%%R
Make_DF = function(TS) {  
    
    d = data.frame(TS$data[, c("abundance","class", "time", "id")])
    d$Sample = rownames(TS$data)


    return(d)
}

In [ ]:
%%R
l = list()
for (i in 1:length(feature)) {
    classname = as.vector(feature[i])
    l[[classname]] = Make_DF(TS[[i]])
    
}

In [ ]:
%%R
DF = do.call(rbind, l)

In [ ]:
%%R
write.csv(DF, 'data/TimeSeries_Abundance_Till_ITS.csv')

In [ ]:
%%R
DF = read.csv('data/TimeSeries_Abundance_Till_ITS.csv')
head(DF)

In [ ]:
%%R
DF2 = tidyr::separate(DF, X, c('OTU', "num", "MY", "Treat", 'Rep'), sep = '\\.' )
DF2$C = paste(DF2$OTU, DF2$num, sep = '.')
DF2$Full.sample = paste(DF2$MY, DF2$Treat, DF2$Rep, sep = '.')
DF2 = filter(DF2, C %in% TS.sig$OTU)
head(DF2)

In [ ]:
%%R
head(DF2)
DF.sum = DF2 %>% group_by(C, time, class) %>% summarise(meanab = mean(abundance), sdab = sd(abundance))

In [ ]:
%%R
head(DF.sum)

In [ ]:
%%R -h 1000 -w 1000
TS.sig$C = TS.sig$OTU

limits = aes(ymax = meanab + sdab, ymin=meanab -sdab, color = class)


ggplot(DF.sum, aes(x = time, y = meanab)) + 
geom_point(aes(color = class)) + 
geom_errorbar(limits, width=0.25)+
geom_segment(data = TS.sig, aes(x = Interval.start, xend = Interval.end, y = 10, yend = 10)) +
facet_wrap(~C)

In [ ]:
%%R -h 400 -w 400
TS.sig$C = TS.sig$OTU

DF.long = filter(DF.sum, C == 'OTU.902')
TS.long = filter(TS.sig, OTU == 'OTU.902')


limits = aes(ymax = meanab + sdab, ymin=meanab -sdab, color = class)


ggplot(DF.long, aes(x = time, y = meanab)) + 
geom_point(aes(color = class)) + 
geom_errorbar(limits, width=0.25)+
geom_segment(data = TS.long, aes(x = Interval.start, xend = Interval.end, y = 10, yend = 10)) +
facet_wrap(~C)

In [ ]:
%%R
(TS.sig) %>% arrange(-Area) %>% head

In [ ]:
%%R -w 1200 -h 500

Mult_TS$ISIE = factor(Mult_TS$ISIE, levels = Mult_TS$ISIE[order(-Mult_TS_sum$counts)])


ggplot(Mult_TS, aes(x = ISIE)) + geom_bar(aes(fill = as.factor(TS_num))) + #+ facet_wrap(~TS_num)+
        theme(text = element_text(size=18),
        axis.title.y = element_text(vjust=1),
        axis.text.x = element_text(angle=90, vjust=0.5, hjust=1))  + xlab('Time Period')+
        ylab('Number of OTUs') 


In [ ]:
%%R
head(TS.sig)
TS.sig$Start[TS.sig$Interval.start %in% c(0:75)] = 'July2014'
#TS.sig$Start[TS.sig$Interval.start > 0 & TS.sig$Interval.start <76] = 'JunetoSept_2014'
TS.sig$Start[TS.sig$Interval.start %in% c(76:110)] = 'September2014'
#TS.sig$Start[TS.sig$Interval.start > 76 & TS.sig$Interval.start < 111 ] = 'SepttoOct_2014'
TS.sig$Start[TS.sig$Interval.start %in% c(111:138)] = 'October2014'
#TS.sig$Start[TS.sig$Interval.start > 111 & TS.sig$Interval.start < 139 ] = 'Oct_Nov_2014'
TS.sig$Start[TS.sig$Interval.start == 139] = 'November2014'
TS.sig$Start[TS.sig$Interval.start > 139 & TS.sig$Interval.start < 294] = 'Nov2014April2015'
TS.sig$Start[TS.sig$Interval.start %in% c(294:341)] = 'April2015'
#TS.sig$Start[TS.sig$Interval.start > 294 & TS.sig$Interval.start < 342] = 'April_June_2015'
TS.sig$Start[TS.sig$Interval.start %in% c(342: 370)] = 'June2015'
#TS.sig$Start[TS.sig$Interval.start > 342 & TS.sig$Interval.start < 371] = 'June_July_2015'
TS.sig$Start[TS.sig$Interval.start %in% c(371: 398)] = 'July2015'
#TS.sig$Start[TS.sig$Interval.start > 371 & TS.sig$Interval.start < 399] = 'July_Aug_2015'
TS.sig$Start[TS.sig$Interval.start %in% c(399: 439)] = 'August2015'
#TS.sig$Start[TS.sig$Interval.start > 399 & TS.sig$Interval.start < 440] = 'Aug_Sept_2015'
TS.sig$Start[TS.sig$Interval.start %in% c(440: 473)] = 'Sept2015'
#TS.sig$Start[TS.sig$Interval.start > 440 & TS.sig$Interval.start < 474] = 'Sept_Oct_2015'
TS.sig$Start[TS.sig$Interval.start %in% c(474: 502)] = 'Oct2015'
#TS.sig$Start[TS.sig$Interval.start > 474 & TS.sig$Interval.start < 503] = 'Oct_Nov_2015'
TS.sig$Start[TS.sig$Interval.start == 503] = 'Nov2015'


TS.sig$End[TS.sig$Interval.end %in% c(0:75)] = 'July2014'
#TS.sig$End[TS.sig$Interval.end > 0 & TS.sig$Interval.end <76] = 'JunetoSept_2014'
TS.sig$End[TS.sig$Interval.end %in% c(76:110)] = 'September2014'
#TS.sig$End[TS.sig$Interval.end > 76 & TS.sig$Interval.end < 111 ] = 'SepttoOct_2014'
TS.sig$End[TS.sig$Interval.end %in% c(111:138)] = 'October2014'
#TS.sig$End[TS.sig$Interval.end > 111 & TS.sig$Interval.end < 139 ] = 'Oct_Nov_2014'
TS.sig$End[TS.sig$Interval.end == 139] = 'November2014'
TS.sig$End[TS.sig$Interval.end > 139 & TS.sig$Interval.end < 294] = 'Nov2014_April2015'
TS.sig$End[TS.sig$Interval.end %in% c(294:341)] = 'April2015'
#TS.sig$End[TS.sig$Interval.end > 294 & TS.sig$Interval.end < 342] = 'April_June_2015'
TS.sig$End[TS.sig$Interval.end %in% c(342: 370)] = 'June2015'
#TS.sig$End[TS.sig$Interval.end > 342 & TS.sig$Interval.end < 371] = 'June_July_2015'
TS.sig$End[TS.sig$Interval.end %in% c(371: 398)] = 'July2015'
#TS.sig$Start[TS.sig$Interval.end > 371 & TS.sig$Interval.end < 399] = 'July_Aug_2015'
TS.sig$End[TS.sig$Interval.end %in% c(399: 439)] = 'August2015'
#TS.sig$End[TS.sig$Interval.end > 399 & TS.sig$Interval.end < 440] = 'Aug_Sept_2015'
TS.sig$End[TS.sig$Interval.end %in% c(440: 473)] = 'Sept2015'
#TS.sig$End[TS.sig$Interval.end > 440 & TS.sig$Interval.end < 474] = 'Sept_Oct_2015'
TS.sig$End[TS.sig$Interval.end %in% c(474: 502)] = 'Oct2015'
#TS.sig$End[TS.sig$Interval.end > 474 & TS.sig$Interval.end < 503] = 'Oct_Nov_2015'
TS.sig$End[TS.sig$Interval.end == 503] = 'Nov2015'





In [ ]:
%%R
Mult_TS = TS.sig %>% group_by(OTU) %>% mutate(TS_num = n()) %>% mutate(diff = Interval.end-Interval.start)

head(Mult_TS) %>% as.data.frame

In [ ]:
%%R
Mult_TS$ISIE = paste(Mult_TS$Start, Mult_TS$End, sep = '-')

In [ ]:
%%R
Mult_TS_sum = Mult_TS %>% group_by(ISIE) %>% summarise(counts = n(), max_diff = max(diff), min_diff = min(diff))

In [ ]:
%%R
Mult_TS$type = ifelse(Mult_TS$diff > 392, 'Long Term',
                      ifelse(Mult_TS$diff %in% c(210: 392), 'Extended',
                            ifelse(Mult_TS$diff %in% c(103: 209), 'Interannual',
                                ifelse(Mult_TS$diff %in% c(30:102), 'Seasonal','Short term'))))

In [ ]:
%%R
Mult_TS %>% group_by(type) %>% filter(Area > 0) %>% summarize(nR = n()) %>% print()
Mult_TS %>% group_by(type) %>% filter(Area < 0) %>% summarize(nH = n()) %>% print

In [ ]:
%%R
tax = as.data.frame(tax_table(physeq.final))
tax$OTU = rownames(tax)
head(tax)

In [ ]:
%%R
tax$Rank2 = gsub("__", "", tax$Rank2)
tax$Rank3 = gsub("__", "", tax$Rank3)
tax$Rank4 = gsub("__", "", tax$Rank4)
tax$Rank5 = gsub("__", "", tax$Rank5)
tax$Rank6 = gsub("__", "", tax$Rank6)
tax$Rank7 = gsub('__', "", tax$Rank7)

In [ ]:
%%R
Mult_TS_tax = left_join(Mult_TS, tax, by = "OTU")

In [ ]:
%%R
Mult_TS_sum_type = Mult_TS_tax %>% group_by(type, Rank2, Rank3, ISIE) %>%
        summarise(counts = n(), max_diff = max(diff), min_diff = min(diff))

In [ ]:
%%R
cols = c("#DE3565",
"#64C036",
"#CA6ED8",
"#B7B938",
"#6F81DD",
"#CE8E2E",
"#528FB8",
"#DD502D",
"#3EBEBD",
"#D44D9C",
"#59C171",
"#AF699B",
"#6E7F27",
"#9781BE",
"#3E814F",
"#C75B6F",
"#BA6040")

In [ ]:
%%R
(Mult_TS_sum_type) %>% group_by(type) %>% summarise(num = sum(counts))

In [ ]:
%%R -w 1000 -h 500


Mult_TS_sum_type$type = factor(Mult_TS_sum_type$type, levels = Mult_TS_sum_type$type[order(-Mult_TS_sum_type$counts)])

p = ggplot(data = Mult_TS_sum_type, aes(type, counts, fill = Rank3)) + geom_bar(stat='identity') +
        theme(text = element_text(size=18), axis.text.x = element_text(angle=0, vjust=1))  + 
        xlab('Temporal Enrichment Pattern') + ylab('Number of OTUs') + scale_fill_manual(values = cols, name = 'Phylum')

p

In [ ]:
%%R -w 1000 -h 500

Longterm = filter(Mult_TS_sum_type, type == 'Long Term')%>% group_by(ISIE, type, Rank2, Rank3) %>%
    summarise(counts = sum(counts)) 
Longterm$ISIE = factor(Longterm$ISIE, levels = Longterm$ISIE[order(-Longterm$counts)])

p = ggplot(data = Longterm, aes(ISIE, counts, fill = Rank3)) + geom_bar(stat='identity') +
        theme(text = element_text(size=16), axis.text.x = element_text(angle=90, vjust=1)) + facet_wrap(~type) +
        scale_fill_manual(values = cols, name = "Phylum")  + xlab(NULL)

p

In [ ]:
%%R -w 1000 -h 500

Mult_TS_sum_type$ISIE = factor(Mult_TS_sum_type$ISIE, levels = Mult_TS_sum_type$ISIE[order(-Mult_TS_sum_type$counts)])

Longterm = filter(Mult_TS_sum_type, type == 'Extended')  %>% group_by(ISIE, type, Rank2, Rank3) %>%
    summarise(counts = sum(counts)) 
Longterm$ISIE = factor(Longterm$ISIE, levels = Longterm$ISIE[order(-Longterm$counts)])

p = ggplot(data = Longterm, aes(ISIE, counts, fill = Rank3)) + geom_bar(stat='identity') +
        theme(text = element_text(size=16), axis.text.x = element_text(angle=90, vjust=1)) + facet_wrap(~type)+
        scale_fill_manual(values = cols, name = "Phylum")   + xlab(NULL)


p

In [ ]:
%%R -w 1000 -h 500


Longterm = filter(Mult_TS_sum_type, type == 'Interannual') %>% group_by(ISIE, type, Rank2, Rank3) %>%
    summarise(counts = sum(counts)) 
Longterm$ISIE = factor(Longterm$ISIE, levels = Longterm$ISIE[order(-Longterm$counts)])

p = ggplot(data = Longterm, aes(ISIE, counts, fill = Rank3)) + geom_bar(stat='identity') +
        theme(text = element_text(size=16), axis.text.x = element_text(angle=90, vjust=1)) + facet_wrap(~type)+
        scale_fill_manual(values = cols, name = "Phylum") + xlab(NULL)


p

In [ ]:
%%R -w 1000 -h 500


Longterm = filter(Mult_TS_sum_type, type == 'Seasonal') %>% group_by(ISIE, type, Rank3) %>%
    summarise(counts = sum(counts)) 
Longterm$ISIE = factor(Longterm$ISIE, levels = Longterm$ISIE[order(-Longterm$counts)])

p = ggplot(data = Longterm, aes(ISIE, counts, fill = Rank3)) + geom_bar(stat='identity') +
        theme(text = element_text(size=16), axis.text.x = element_text(angle=90, vjust=1)) + facet_wrap(~type)+
        scale_fill_manual(values = cols, name = "Phylum") + xlab(NULL)


p

In [ ]:
%%R
head(Mult_TS) %>% as.data.frame()
o = filter(Mult_TS, TS_num == 1)
one = filter(diff, OTU %in% o$OTU)

In [ ]:
%%R
one$Start_stop = paste(one$Start, one$End, sep = '__')
head(one)

In [ ]:
%%R -h 800 -w 1000
plot = one %>% group_by(Start_stop) %>% summarise(counts = n())
plot$Start_stop = factor(plot$Start_stop, levels = plot$Start_stop[order(-plot$counts)])

ggplot(data = plot, aes(x = Start_stop, y = counts)) + geom_bar(stat = 'identity') +
        theme(text = element_text(size=18),
        axis.title.y = element_text(vjust=1),
        axis.text.x = element_text(angle=90, vjust=0.5, hjust=1))  + xlab('Time Period')+
        ylab('Number of OTUs') 

In [ ]:
%%R
head(one)

In [ ]:
%%R
head(one)

one %>% filter(diff >=503) %>% summarise(totallyconsistent = n()) %>% print()
one %>% filter(diff >=384) %>% summarise(longterm = n()) %>% print()
one %>% filter(diff >=35 & diff <=76) %>% summarise(shortterm = n()) %>% print()
one %>% filter(diff <35) %>% summarise(month = n()) %>% print()
one %>% filter(diff >35 & diff <= 120) %>% summarise(medium = n()) %>% print()
one %>% filter(diff >120 & diff < 400) %>% summarise(medium = n()) %>% print()

In [ ]:
%%R
med = one %>% filter(diff >35 & diff <= 120) 
head(med)


In [ ]:
%%R
NTR.bulk = read.csv('data/NTR_ApNov_Enriched.csv')
med.ab = filter(DF.sum, C %in% NTR.bulk$OTU)
med.sig = filter(TS.sig, C %in% NTR.bulk$OTU)

In [ ]:
%%R
length(unique(NTR.bulk$OTU))

In [ ]:
%%R
head(TSOM)

In [ ]:
%%R
filter(TSOM, X == 'OTU.10 interval:1')

In [ ]:
%%R -h 400 -w 800

limits = aes(ymax = meanab + sdab, ymin=meanab -sdab, color = class)

ggplot(med.ab, aes(x = time, y = meanab)) + 
geom_point(aes(color = class)) + 
geom_errorbar(limits, width=0.25)+
geom_segment(data = med.sig, aes(x = Interval.start, xend = Interval.end, y = 10, yend = 10)) +
facet_wrap(~C)

In [ ]:
%%R
head(med.ab)

In [ ]:
%%R
test = one %>% filter(diff >=384, Area >0) %>% summarise(.resp = n()) %>% print
test = one %>% filter(diff >=384, Area <0) %>% summarise(NT.resp = n()) %>% print

In [ ]:
%%R
sessionInfo()